In [1]:
# Import modules.
import datetime
import numpy as np
import random
from random import shuffle
from geopy.distance import vincenty

from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Statistics
from pyevolve import Mutators
from pyevolve import Selectors

In [2]:
# Fitness function.
# It simply calculates the distance in kilometers of the current path.
def fitness(chromosome):
    score = 0
    for i in range(len(chromosome)):
        j = (i + 1) % len(chromosome)
        score += matrix[chromosome[i], chromosome[j]]
    return score

In [17]:
it = 0
def getIndex(list_i, elem):    
    i = 0
    lisst = list_i.genomeList
    for value in lisst :
        if (value == elem):
            return i
        i = i + 1

# Crossover Operator CX2
def G1DListCX2(genome, **args):
    global it
    it += 1
    sister   = None
    brother  = None
    gMom = args['mom'].clone()
    gDad = args['dad'].clone()
    print(args)
    if len(gMom) == 1:
        Util.raiseException("The Binary String have one element, can't use the Two Point Crossover method !", TypeError)

    O2  = gDad.clone()
    O1   = gDad.clone()
    
    for i in range(14):
        O1[i] = 0
        O2[i] = 0
 
    idx1 = 0
    idx2 = 0
    O1[idx1] = gDad[idx1]
    
    
    while(idx1 < len(gDad)):
        i0             = getIndex(gMom, O1[idx1])
        temp           = gDad[i0]
        i0             = getIndex(gMom, temp)
        O2[idx2]  = gDad[i0]
        idx1          +=  1
        if(not(idx1 < len(gDad))):
            break
        i1            = getIndex(gMom, O2[idx2])
        temp          = gDad[i1]
        
        O1[idx1]  = temp
        idx2          += 1
            
        print('sister',O1.genomeList)
        print('brother',O2.genomeList)
        
    print('indexes', idx1, idx2)
    return O1, O2
        

In [4]:
# Callback called each time for each generation.
def tsp_callback(ga_engine):
    stats.append(ga.getStatistics().asTuple())
    return False

In [9]:
# Initializator for the 1DList.
# Simply creates a list containing the cities indexes and shuffle it.
def list_initializator(genome, **args):
    cities = [i for i in range(len(coords))]
    random.shuffle(cities)
    genome.genomeList = cities
    #   genome.setInternalList(cities)


In [6]:
# Save the GA statistics in `filename`.
def save_stats(filename, stats):
    header = 'rawMax;rawMin;rawAve;rawDev;rawVar;fitMax;fitMin;fitAve'
    np.savetxt(filename, stats, '%.2f', delimiter=';', header=header, comments='')
    print('Saved stats into ' + filename)

In [18]:
# GPS position of each city.
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

# Create a 1D list of tuples with latitude and longitude for each city.
coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))

# Create a distance matrix between each city.
matrix = {}
for i, city1 in enumerate(coords):
    for j, city2 in enumerate(coords):
        dist = vincenty(city1, city2).km
        matrix[i, j] = dist
        
# Constants.
POPULATION_SIZE = 100
MUTATION_RATE   = 0.01
CROSSOVER_RATE  = 0.9
GENERATIONS_NB  = 300
CROSSOVER_FUNC  = G1DListCX2
#CROSSOVER_FUNC  = Crossovers.G1DListCrossoverEdge

# Array for collecting statistics.
stats = []

# The chromosome is represented by a 1DList which contains the cities indexes [0,13]
genome = G1DList.G1DList(len(coords))
# Set fitness function.
genome.evaluator.set(fitness)
# Set crossover function.
genome.crossover.set(CROSSOVER_FUNC)
# Set initializator function.
genome.initializator.set(list_initializator)

# GA initialization.
ga = GSimpleGA.GSimpleGA(genome)

ga.setPopulationSize(POPULATION_SIZE)
ga.setMutationRate(MUTATION_RATE)

ga.setCrossoverRate(CROSSOVER_RATE)
# Elitism is important for TSP.
ga.selector.set(Selectors.GTournamentSelector)

ga.setElitism(True)
# We aim for minimization.
ga.setMinimax(Consts.minimaxType["minimize"])

# Number of generations
ga.setGenerations(GENERATIONS_NB)

# In case we want to monitor the evolution process
# execute the function current_best every generation
ga.stepCallback.set(tsp_callback)

ga.evolve(freq_stats=500)

# Final best solution
print(ga.bestIndividual())

# Save statistics.
if CROSSOVER_FUNC == Crossovers.G1DListCrossoverEdge:
    crossover_name = 'edge'
else:
    crossover_name = 'perso'
    
now = datetime.datetime.now()

save_stats('stats/' +
               crossover_name +
               '_pop' + str(POPULATION_SIZE) +
               '_gen' + str(GENERATIONS_NB) +
               '_' + now.strftime('%d%m%Y-%H%M') +
               '.csv',
           stats)

# Best path found: [1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9, 0]
# For crossovers inspiration: https://www.hindawi.com/journals/cin/2017/7430125/

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [8062.00(8064.29)/4922.15(4919.09)/6718.33(6718.33)]
{'mom': - GenomeBase
	Score:			 5885.252734
	Fitness:		 5886.670742

	Params:		 {}

	Slot [Evaluator] (Count: 1)
		Name: fitness - Weight: 0.50
	Slot [Initializator] (Count: 1)
		Name: list_initializator - Weight: 0.50
	Slot [Mutator] (Count: 1)
		Name: G1DListMutatorSwap - Weight: 0.50
		Doc:  The mutator of G1DList, Swap Mutator

   .. note:: this mutator is :term:`Data Type Independent`

   
	Slot [Crossover] (Count: 1)
		Name: G1DListCX2 - Weight: 0.50

- G1DList
	List size:	 14
	List:		 [13, 3, 5, 11, 1, 4, 6, 10, 9, 2, 8, 7, 0, 12]

, 'dad': - GenomeBase
	Score:			 5869.173302
	Fitness:		 5870.618679

	Params:		 {}

	Slot [Evaluator] (Count: 1)
		Name: fitness - Weight: 0.50
	Slot [Initializator] (Count: 1)
		Name: list_initializator - Weight: 0.50
	Slot [Mutator] (Count: 1)
		Name: G1DListMutatorSwap - Weight: 0.50
		Doc:  The mutator of G1DList, Swap Mutator

   .. note:: this mutator 

TypeError: list indices must be integers or slices, not NoneType